# コード7-1：偏微分方程式の数値計算の実装

『ウイルス感染の数理モデルとシミュレーション ーデータを定量的に理解するー』

偏微分方程式の数値計算の実装の例をまとめる。

## ライブラリ、パッケージ（モジュール）の読み込み

In [1]:
## 必要があればインストール
# install.packages("ggplot2", "patchwork", "deSolve)

In [2]:
rm(list = ls(all = TRUE))
library(ggplot2)
library(patchwork)
library(deSolve)

## 偏微分方程式の数値計算

式(7.2)

$$
\frac{d T(t)}{dt} =
$$

等価な式(7.5)を実験データに合わせて変形した式(7.6)

$$
\frac{d T(t)}{dt} =
$$

$V(t) = V_I(t) + V_{NI}(t), V_\theta(t) = \alpha V_I(t), \beta_\theta = \frac{\beta}{\alpha}, f_\theta = \alpha f$であるので

式(7.2)を以下のように変形する。

$$
\frac{d T(t)}{dt} = 
$$

式(7.2)を変形した偏微分方程式と式(7.5)の常微分方程式をそれぞれ数値計算することで比較する。